### Text Generation Using RNN trained without any pretrained embeddings and RNN trained with Word2Vec cbow embedding
In this notebook, we will compare the performance of two models: an RNN model trained directly wthout any pretrained embeddings and another RNN model using pretrained embeddings. We will analyze the generated text using three primary metrics: **perplexity**, **diversity**, and **semantic coherence**.

### Setup
### Import Libraries and Suppress Warnings

In [1]:
import pickle
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
import warnings
from keras.models import load_model
from keras.layers import BatchNormalization
from keras.utils import get_custom_objects
import tensorflow as tf
import absl

# Suppress TensorFlow and Python warnings for cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress all logs except FATAL
tf.get_logger().setLevel('ERROR')
absl.logging.set_verbosity(absl.logging.ERROR)
warnings.filterwarnings('ignore')

### Load the Necessary Data and Models

In [2]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

# Load the necessary word-to-index and index-to-word dictionaries
with open('pickle/word_to_index_ps.pkl', 'rb') as file:
    word_to_index = pickle.load(file)

with open('pickle/index_to_word_ps.pkl', 'rb') as file:
    index_to_word = pickle.load(file)

### Function to Generate Text from a Start Prompt
This function generates text using a given model and also collects probabilities for analysis.

In [3]:
# Function to Generate Text from a Start Prompt and Collect Probabilities
def generate_sentence_with_probabilities(model, word_to_index, index_to_word, max_seq_length, start_text, senten_max_length):
    start_text = start_text.lower()  # Convert start_text to lowercase
    new_sentence = [word_to_index.get(word, word_to_index[unknown_token]) for word in start_text.split()]
    probabilities = []  # List to store probabilities of chosen words

    # Generate text until SENTENCE_END token or max length is reached
    while len(new_sentence) < senten_max_length:
        sequence = new_sentence
        sequence_padded = pad_sequences([sequence], maxlen=max_seq_length, padding='pre')
        predicted_probs = model.predict(sequence_padded, verbose=0)[0]

        # Sample the next word index using the probabilities
        sampled_word_index = np.random.choice(len(predicted_probs), p=predicted_probs)
        probabilities.append(predicted_probs[sampled_word_index])  # Store the probability

        if sampled_word_index == word_to_index.get(sentence_end_token):
            break
        new_sentence.append(sampled_word_index)

    # Convert indices back to words
    sentence_str = [index_to_word[idx] for idx in new_sentence]
    generated_text = ' '.join(sentence_str)
    generated_text = generated_text.replace('_ comma _', ',').replace('_ eol _', '.\n') + '.'
    return generated_text, probabilities

### Load both the models and generate sentences

In [4]:
# Load and generate text using RNN model
get_custom_objects().update({'BatchNormalization': BatchNormalization})
rnn_model = load_model('model/rnn_direct_model.h5', custom_objects={'BatchNormalization': BatchNormalization})

# Generate text using RNN model
rnn_sentences = []
rnn_probabilities = []

print("=== Sentences Generated by RNN Model without any pretrained embedding===\n")
for i in range(11):
    start_prompt = "குந்தவை தேவி"
    generated_output, probs = generate_sentence_with_probabilities(
        rnn_model, word_to_index, index_to_word, 75, start_prompt, senten_max_length=50
    )
    rnn_sentences.append(generated_output)
    rnn_probabilities.extend(probs)
    print(f"Generated text - {i}:", generated_output)

=== Sentences Generated by RNN Model without any pretrained embedding===

Generated text - 0: குந்தவை தேவி பார்த்துக் கொண்டிருப்பது போல மக்கள் அந்தக் உரமும் உள்ள இரண்டு பேர் என்னைத் தோள் கொடுத்துத் தூக்கிக் கொள்ளுங்கள் நகர்ந்து வந்து ஒரு சத்தம் வந்து நிற்பதைக் கண்டு சிந்தனையில் தாதியரின் முகங்களும் உடனே போகிறான் ராணிக்கு பயங்கரமாக விட்டேன்.
Generated text - 1: குந்தவை தேவி வானதியைப் தான் பார்த்து அது பல்லக்கில் இருந்தது என்பதை முன்னமே நான் ஒரு நாளும் சொல்ல மாட்டேன் தாங்கள் வெளியில் செல்ல இளவரசிகள் இருவரும் ஆதுர சென்று சாலையில் சோழப் பேரரசில் பெருந்தரத்து அரசாங்க அதிகாரிகளாகவும் பதவி வகித்து வந்தார்கள் என்பது தவறி விழுந்து முழுகி இறந்து விட்டார் என்று சொன்னாய்.
Generated text - 2: குந்தவை தேவி என்னிடம் கூறியது முடியாத போது யார் என்பது மதுராந்தகர்.
Generated text - 3: குந்தவை தேவி அவருடைய முகத்தையே பார்த்துக் கொண்டு வந்தியத்தேவனும் சிறிது தூரம் சென்று கொண்டிருந்தார்கள் என்று பார்த்துக் கொள்ளுங்கள் சண்டை எதற்கு.
Generated text - 4: குந்தவை தேவி இலங்கைக்குப் போகிறேன் என்று சொல்லிக் கொண்டூ 

In [5]:
# Load and generate text using RNN CBOW model
cbow_model = load_model('model/rnn_cbow_model.h5', custom_objects={'BatchNormalization': BatchNormalization})

# Generate text using CBOW model
cbow_sentences = []
cbow_probabilities = []

print("\n=== Sentences Generated by RNN Model with Word2Vec CBOW pretrained embeddings===\n")
for i in range(11):
    start_prompt = "குந்தவை தேவி"
    generated_output, probs = generate_sentence_with_probabilities(
        cbow_model, word_to_index, index_to_word, 75, start_prompt, senten_max_length=50
    )
    cbow_sentences.append(generated_output)
    cbow_probabilities.extend(probs)
    print(f"Generated text - {i}:", generated_output)


=== Sentences Generated by RNN Model with Word2Vec CBOW pretrained embeddings===

Generated text - 0: குந்தவை தேவி முதலியோர் வந்தியத்தேவனைப் பார்த்து விட்டு வருவதாகச் சொல்லிவிட்டு விடு என்று சொல்ல வேண்டும் என்ற ஆசை தானே ஆசை கொண்டு சின்னப் பழுவேட்டரையரின் மனதில் கொஞ்ச தூரம் வல்லவரையனுக்கு காதில் ஒன்று விழுந்து வேளக்காரப் படையில் ஓர் எண்ணத்தை அங்கே வந்தியத்தேவன் அங்கே வந்து மணம் புரிந்த மருமகர் திடீரென்று அவன் பின் அபிமானம் இருந்தது என்று எனக்குப் அவள் மனத்தில் தெரிந்து கொண்டூ தான் அங்கே.
Generated text - 1: குந்தவை தேவி கரையை கீழே கிடந்த மீது கால் ஒரு கை பார்க்க வேண்டும் என்று வானத்தைப் ஆட்சி தெரிந்து கொள்வது பின்னர் அந்த மகா மன்னர் பரம்பரையில் வந்து சில தூரத்தில் பகைவர்கள் வந்தவர்கள் அல்லவா பொன் கோயில் எடுக்க வேண்டும் என்றும் சொல்லி விட்ட மனம் நிம்மதி வந்தியத்தேவனுடைய மனதில் ஏரி ஏரி என்று பெயர் மாறுதல் யோசித்துக் கொண்டிருந்தார்கள் என்று முடிவு செய்து கொண்டிருந்தபோது மனம்.
Generated text - 2: குந்தவை தேவி அனுப்பி விட்டூ மறு மீது யாரும் குற்றம் திருப்தி செய்ய வேண்டும்” என்றார் குந்தவை த

In [6]:
# Load and generate text using RNN CBOW model
dnn_model = load_model('model/dnn_model.h5')
# Generate text using CBOW model
dnn_sentences = []
dnn_probabilities = []

print("\n=== Sentences Generated by DNN Model without any pretrained embedding===\n")
for i in range(11):
    start_prompt = "குந்தவை தேவி"
    generated_output, probs = generate_sentence_with_probabilities(
        dnn_model, word_to_index, index_to_word, 75, start_prompt, senten_max_length=50
    )
    dnn_sentences.append(generated_output)
    dnn_probabilities.extend(probs)
    print(f"Generated text - {i}:", generated_output)


=== Sentences Generated by DNN Model without any pretrained embedding===

Generated text - 0: குந்தவை தேவி ஏமாற்றி நிறுத்தி பார்த்த காரியத்தை அவர் உன்னைத் திரும்பிப் பார்த்து சொல்லிக் கொண்டிருந்தான் போதும் என்பதை தெரிந்து கொள்ள குந்தவை , பூர்வீக நதியில் ஒரு கதை கீழே மதி , எவ்வளவு கதை UNKNOWN_TOKEN , அந்த மிருகங்களையெல்லாம் பிடித்துக் UNKNOWN_TOKEN , சொர்க்கத்திலிருந்து UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN இல்லை.
Generated text - 1: குந்தவை தேவி தாங்களும் தப்பித்துக் கொண்டு போகும் போது சேவை UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN UNKNOWN_TOKEN குறுக்கு வழியில் UNKNOWN_TOKEN கை UNKNOWN_TOKEN UNKNOWN_TOKEN அதனால்.
Generated text - 2: குந்தவை தேவி அவனிடம் என்னைப் பிரிந்து வந்து ஓரிடத்தில் வந்து , பழுவேட்டரையர் பெயரால் மாறாக வந்தியத்தேவன் நடந்த நீ என்ன எதிர்ப்பட்டால் UNKNOWN_TOKEN கொள்ள வேண்டும்.
Generated text - 3: குந்தவை தேவி இங்கே வந்தியத்தேவனும் சுரங்கப் பாதையில் அதன் பல்லக்கு தரையில் விழாமல் மோதிக் கொண்டு ஒரு விஷயம் என்பதையும் தெரிந்துகொள்ள.
Generated text - 4: குந்தவை தேவி வந

### Perplexity
Perplexity measures how well a probability model predicts a sample. Lower values indicate better performance.

In [7]:
# Function to calculate Perplexity
def calculate_perplexity(probabilities):
    return np.exp(-np.mean(np.log(probabilities))) if probabilities else float('inf')

# Calculate perplexity for both models
rnn_perplexity = calculate_perplexity(rnn_probabilities)
cbow_perplexity = calculate_perplexity(cbow_probabilities)
dnn_perplexity = calculate_perplexity(dnn_probabilities)
print(f"RNN-direct embedding Model Perplexity: {rnn_perplexity:.2f}")
print(f"RNN-cbow embedding Model Perplexity: {cbow_perplexity:.2f}")
print(f"DNN-direct embedding Model Perplexity: {dnn_perplexity:.2f}")

RNN-direct embedding Model Perplexity: 3.97
RNN-cbow embedding Model Perplexity: 13.47
DNN-direct embedding Model Perplexity: 25.08


### Diversity
Diversity measures the uniqueness of words used in the generated text. Higher values indicate more diverse vocabulary.

In [8]:
# Function to calculate Diversity
def calculate_diversity(sentences):
    unique_words = set()
    total_words = 0
    for sentence in sentences:
        words = sentence.split()
        unique_words.update(words)
        total_words += len(words)
    return len(unique_words) / total_words if total_words > 0 else 0

# Calculate diversity for both models
rnn_diversity = calculate_diversity(rnn_sentences)
cbow_diversity = calculate_diversity(cbow_sentences)
dnn_diversity = calculate_diversity(dnn_sentences)
print(f"RNN-direct embedding Model Diversity: {rnn_diversity:.2f}")
print(f"RNN-cbow embedding Model Diversity: {cbow_diversity:.2f}")
print(f"DNN-direct embedding Model Diversity: {dnn_diversity:.2f}")

RNN-direct embedding Model Diversity: 0.77
RNN-cbow embedding Model Diversity: 0.64
DNN-direct embedding Model Diversity: 0.70


### Semantic Coherence
Semantic coherence measures how well the words in a sentence relate to each other. Higher values indicate better coherence.

In [9]:
# Function to calculate Semantic Coherence
from sklearn.metrics.pairwise import cosine_similarity

# Function to calculate Semantic Coherence
def calculate_semantic_coherence(sentences, model, word_to_index):
    embedding_layer = model.get_layer('embedding_layer')
    embedding_weights = embedding_layer.get_weights()[0]

    sentence_vectors = []
    for sentence in sentences:
        words = sentence.split()
        vectors = [embedding_weights[word_to_index[word]] for word in words if word in word_to_index]
        if vectors:
            sentence_vector = np.mean(vectors, axis=0)
            sentence_vectors.append(sentence_vector)

    sentence_vectors = np.array(sentence_vectors)
    pairwise_similarities = cosine_similarity(sentence_vectors)
    avg_similarity = np.mean(pairwise_similarities)
    return avg_similarity

# Calculate semantic coherence for both models
rnn_coherence = calculate_semantic_coherence(rnn_sentences, rnn_model, word_to_index)
cbow_coherence = calculate_semantic_coherence(cbow_sentences, cbow_model, word_to_index)
print(f"RNN-direct embedding Model Semantic Coherence: {rnn_coherence:.2f}")
print(f"RNN-cbow embedding Model Semantic Coherence: {cbow_coherence:.2f}")

RNN-direct embedding Model Semantic Coherence: 0.27
RNN-cbow embedding Model Semantic Coherence: 0.50


## Final Results and Conclusion

1. **`Perplexity`**: The RNN model without pre-trained embeddings achieved a lower perplexity (3.97) compared to the CBOW-embedding RNN model (13.47), with the DNN model scoring the highest perplexity at 25.08. This indicates that the direct RNN model is more effective at predicting sequences within this dataset, making it better suited for generating text with more accurate predictions in this specific context.

2. **`Diversity`**: The RNN model without pre-trained embeddings demonstrated the highest diversity (0.77), closely followed by the direct DNN model (0.70) and the CBOW model (0.64). This shows that both RNN and DNN models generated a more varied vocabulary. The direct RNN model shows a strong ability to produce diverse language while maintaining contextual relevance, out of all.

3. **`Semantic Coherence`**: As expected, the RNN model with CBOW embeddings outperformed in terms of semantic coherence, achieving a higher average coherence score (0.50) compared to the direct RNN model (0.27). This aligns with the purpose of using CBOW embeddings, which are designed to capture meaningful relationships between words and boost coherence in generated text. The DNN model was not evaluated for semantic coherence, as it lacks the architecture to capture semantic relationships effectively, and high training loss further limited its ability to produce semantically structured sentences.

`Based on these metrics`:
- For **accurate predictions and vocabulary diversity**, the RNN model trained without pre-trained embeddings is a strong choice, with low perplexity and high diversity, making it suitable for scenarios where variety and predictive accuracy are prioritized.
- For **better semantic coherence**, the RNN model with CBOW embeddings is preferred, as it benefits from pre-trained embeddings that enhance cohesion in generated text.

### Final Recommendation
While pre-trained embeddings typically enhance semantic coherence, in this case, the **RNN model without pre-trained embeddings is preferable**. It provides high predictive accuracy, meaningful diversity, and an overall coherent style that aligns closely with the Ponniyin Selvan narrative. However, if semantic coherence is prioritized, the CBOW model remains a valuable alternative.